# Introduction

This notebook implements a simple model of transportation emissions to help evaluate different ways of getting to the mountains in Switzerland.

# Data sources

Primary data sources for this model are:

* [Mobitool v3.0](https://www.mobitool.ch/), Paul Scherrer Institut. 2023.
* Sacchi and Bauer, [Understanding the carbon footprint of your daily commute: a guide to climate-friendly urban transport](https://blogs.ethz.ch/energy/daily-commute-carbon-footprint/), Energy Blog, ETH Zurich, 2023;
* UK Department for Energy Security and Net Zero [Greenhouse gas reporting: conversion factors 2023](https://www.gov.uk/government/publications/greenhouse-gas-reporting-conversion-factors-2023) (assumes UK energy mix for electric vehicles, which is dirtier than the Swiss energy mix);
* [Our World in Data](https://ourworldindata.org/travel-carbon-footprint) (based on the 2022 UK report).

Other data sources for sanity checking:
* manufacturer data on electric car efficiency (https://www.evspecs.org/);
* CH Bundesamt für Energie [CO2 emissions regulations](https://www.bfe.admin.ch/bfe/en/home/efficiency/mobility/co2-emission-regulations-for-new-vehicles.html) for new vehicles;
* [Electricity Maps](https://app.electricitymaps.com/map).

PSI numbers include full lifecycle (manufacture and disposal), UK numbers do not.


In [14]:
gco2e_per_km = {
    # g CO2e / pkm [ OWID, PSI]. 0 means no data.
    # Road transport. For bus, rail, and tram, PSI numbers assume 30% load factor.
    "bicycle": [0, 6],
    "coach bus": [27, 57],
    "gasoline car": [170, 242],
    "electric car": [47, 110],
    "motorbike": [114, 163],
    # Rail transport
    "tgv": [4, 21],
    "rail UK": [35, 0],
    "rail CH": [0, 7],
    "tram": [29, 43],
    # Air transport. PSI numbers assume ~80% load factor
    "flight short": [246, 292],  # continental, economy
    "flight long": [253, 285],   # intercontinental, economy
}

heli_kg_co2e_per_h = 966.489
kgco2e_per_km = {k: [v[0] / 1000, v[1] / 1000] for k, v in gco2e_per_km.items()}
kgco2e_per_km

{'bicycle': [0.0, 0.006],
 'coach bus': [0.027, 0.057],
 'gasoline car': [0.17, 0.242],
 'electric car': [0.047, 0.11],
 'motorbike': [0.114, 0.163],
 'tgv': [0.004, 0.021],
 'rail UK': [0.035, 0.0],
 'rail CH': [0.0, 0.007],
 'tram': [0.029, 0.043],
 'flight short': [0.246, 0.292],
 'flight long': [0.253, 0.285]}

In [19]:
from ipywidgets import interactive_output, IntSlider, Label, Output, RadioButtons, VBox
import matplotlib.pyplot as plt
from IPython.display import display, HTML

style = {'description_width': 'initial'}
layout = {'width': '400px'}
radio_model = RadioButtons(options=['OWID', 'PSI'], value='PSI', description='Model:', style=style, layout=layout)
slider_distance_km = IntSlider(value=200, min=0, max=400, step=10, description='Distance (km)', style=style, layout=layout, continuous_update=False)
slider_car_passengers = IntSlider(value=1, min=1, max=5, step=1, description='Car passengers', style=style, layout=layout, continuous_update=False)
output = Output()

def recalc(model, distance_km, car_passangers):
    idx = 0 if model == 'OWID' else 1
    data = {
        "bicycle": kgco2e_per_km["bicycle"][idx] * distance_km,
        "coach bus": kgco2e_per_km["coach bus"][idx] * distance_km,
        "gasoline car": kgco2e_per_km["gasoline car"][idx] * distance_km / car_passangers,
        "electric car": kgco2e_per_km["electric car"][idx] * distance_km / car_passangers,
        "motorbike": kgco2e_per_km["motorbike"][idx] * distance_km,
        "rail CH": kgco2e_per_km["rail CH"][idx] * distance_km,
    }
    with output:
        table_html = "<table style='border-collapse: collapse'>"
        table_html += "<tr><th style='border: 1px solid black;'>Mode </th><th style='border: 1px solid black;'>kg CO2e / passenger</th></tr>"
        for k, v in data.items():
            table_html += f"<tr><td style='border: 1px solid black;'>{k}</td><td style='border: 1px solid black;'>{v:.1f}</td></tr>"
        table_html += "</table>"
        output.clear_output()
        display(HTML(table_html))
        
        labels = data.keys()
        values = [data[k] for k in labels]
        
        plt.figure(figsize=(8, 5))
        plt.bar(labels, values, color='skyblue')
        plt.xlabel('Mode')
        plt.ylabel('kg CO2e / person')
        plt.ylim(0, 100)
        #plt.yscale('log')
        plt.grid(axis='y')
        plt.show()

out = interactive_output(recalc, {
    'model': radio_model,
    'distance_km': slider_distance_km,
    'car_passangers': slider_car_passengers
})

display(VBox([radio_model, slider_distance_km, slider_car_passengers, out, output]))
